In [142]:
! pip install --quiet fuzzywuzzy[speedup]
! pip install --quiet toolz

In [194]:
import pandas as pd
import numpy as np
import json
from validation.title_matching import *

In [54]:
direct_match = (pd
                .read_excel('./data/SOC/soc_2010_direct_match_title_file.xls', skiprows=6)
                .rename(columns = {'2010 SOC Direct Match Title': 'title', 
                                     '2010 SOC Code': 'code'})[['title', 'code']])

samples = (pd
           .read_excel('./data/SOC/Sample of Reported Titles.xlsx')
           .rename(columns = {'Reported Job Title': 'title', 
                              'O*NET-SOC Code': 'code'})[['title', 'code']])

alternates = (pd
              .read_excel('./data/SOC/Alternate Titles.xlsx')
              .rename(columns = {'Alternate Title': 'title', 
                                 'O*NET-SOC Code': 'code'})[['title', 'code']])

alternates['code'] = alternates.code.str.slice(0,7)
samples['code'] = samples.code.str.slice(0,7)

In [198]:
BAD_TITLES = ['assistant']

lookup = pd.concat([direct_match, samples, alternates]).drop_duplicates(['code', 'title'])
lookup['title'] = lookup.title.str.lower()

# filter all titles that have more than one match with different codes!
idx = lookup.groupby('title').transform('count').code
lookup = lookup[idx == 1] 

lookup = lookup[~lookup.title.isin(BAD_TITLES)]

In [199]:
lookup.to_csv('crosswalks/soc-title-lookup.csv', index=False)

In [ ]:
with open('./data/sample.jl', 'r') as f:
    jobs = pd.DataFrame([json.loads(d) for d in f])
jobs['title'] = jobs.title.str.lower()

In [195]:
matcher = layered_matcher([exact_matcher(lookup), 
                           title_matcher(lookup, punct_lookup(lookup))])

In [196]:
matched = matcher(jobs)

In [197]:
matched.shape[0] / jobs.shape[0]

0.194